In [81]:
from azureml.core import Workspace, Dataset

subscription_id = '20d4fdf3-6a4b-4f0b-a842-bd7392136332'
resource_group = 'cienciadatos'
workspace_name = 'azureml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_id(workspace, '9227d1d4-d528-48db-84ed-768975adbf63')
dataset.download(target_path='.', overwrite=False)

['c:\\Users\\pablo.tamayo\\Desktop\\DataSchool\\Azure\\AzureDataSchool\\train_data.csv']

In [7]:
import pandas as pd

df = pd.read_csv('train_data.csv', delimiter=',')


In [10]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from azureml.core import Run
import pandas as pd
import numpy as np
import argparse
import os
import logging

x = df.drop('Class', axis=1, inplace=False)
y = df['Class']

'''
lr_cv = cross_validate(LogisticRegression(max_iter=400), x, y, cv=2, scoring=['balanced_accuracy', 'recall', 'precision'], return_train_score=True)
print('Listo regresion lineal')
#rf_cv = cross_validate(RandomForestClassifier(), x, y, cv=3, scoring=['balanced_accuracy', 'recall', 'precision'], return_train_score=True)
#print('Listo random forest')
svm = cross_validate(SVC(), x, y, cv = 3, scoring=['balanced_accuracy', 'recall', 'precision'], return_train_score=True)
print('Listo SVM')

hola =[]
for score_model, name_model in [(lr_cv, 'lr'), (svm, 'svm')]:
    hola.append({
        f'train_acc_{name_model}': np.mean(score_model['train_balanced_accuracy']),
        f'train_recall_{name_model}': np.mean(score_model['train_recall']),
        f'train_precision_{name_model}': np.mean(score_model['train_precision']),
        f'test_acc_{name_model}': np.mean(score_model['test_balanced_accuracy']),
        f'test_recall_{name_model}': np.mean(score_model['test_recall']),
        f'test_precision_{name_model}': np.mean(score_model['test_precision'])
    })


'''

param_dist ={
    'penalty':['l1', 'l2', 'elasticnet'],
    'C': [c/10 for c in range(1,20,5)],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

}
clf = RandomizedSearchCV(LogisticRegression(), param_dist, n_iter=5, random_state=0, refit='balanced_accuracy')
'''
mlflow.log_params({
    'penalty': 'l2',
    'C': 1.0,
    'solver':'lbfgs'
})
'''
clf.fit(x,y)

#mlflow.sklearn.log_model(lr, artifact_path='skleran-model')
y_pred = clf.predict(x)

recall = recall_score(y,y_pred)
precision = precision_score(y, y_pred)
acc = accuracy_score(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)
print(recall)
print(precision)
print(acc)
print(conf_matrix)

#logging.info(f'matriz de confusion{conf_matrix}')

'''
mlflow.log_metrics({
    'recall':recall,
    'precision': precision,
    'accuracy': acc,

})
'''

c:\Users\pablo.tamayo\Anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\pablo.tamayo\Anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\pablo.tamayo\Anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\pablo.tamayo\Anaconda3\envs\myenv\lib\site-packages\sklearn\linear_mo

0.6371029795237891
0.6562854343494391
0.6458278994957398
[[59238 31222]
 [33957 59615]]


"\nmlflow.log_metrics({\n    'recall':recall,\n    'precision': precision,\n    'accuracy': acc,\n\n})\n"

In [12]:
clf.best_params_

{'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.6}